In [1]:
import pandas as pd
import os

In [2]:
Desktop = os.path.join(os.path.expanduser("~"), "Desktop")

filepath_root = os.path.join(Desktop, "LBP_Revision")

os.path.exists(filepath_root)

path_inclusion_crit_LBP = os.path.join(filepath_root,'inclusion_codes_LBP.xlsx')

path_inclusion_crit_scoliosis = os.path.join(filepath_root,'inclusion_codes_scoliosis.xlsx')


In [3]:
def get_cpts_fromfile(filepath=''):
    df_cpt = pd.read_excel(filepath, dtype=str, sheet_name='CPT Codes')
    return df_cpt['CPT Code']       

def get_icds_fromfile(filepath=''):        
    df_icd = pd.read_excel(filepath,dtype=str, sheet_name='ICD Codes')
    return df_icd['ICD Code']

In [4]:
LBP_filename = 'LBP_all'
scoliosis_filename = 'scoliosis_all'
LBP_all_data = pd.read_csv(filepath_root + '/' + LBP_filename + '.csv', encoding = 'utf8',dtype=str)
scoliosis_all_data = pd.read_csv(filepath_root + '/' + scoliosis_filename + '.csv', encoding = 'utf8',dtype=str)

In [5]:
print(LBP_all_data.keys())


Index(['EMPI', 'MGH_MRN', 'Date.prc', 'Date.dia', 'Code_Type.prc', 'Code.prc',
       'Code_Type.dia', 'Code.dia', 'Diagnosis_Name', 'Procedure_Name',
       'Gender_Legal_Sex', 'Age', 'Language', 'Race1', 'Marital_status',
       'Religion', 'Date_Of_Death', 'AgeAtPrc', 'Admit_Date', 'Discharge_Date',
       'Discharge_Disposition', 'diagnosis_list', 'Diagnosis_list_combined',
       'Diagnosis_list_no_NA', 'City', 'State', 'Zip', 'Country',
       'Insurance_1', 'Insurance_2', 'Insurance_3', 'Phone_Code',
       'Physical_Date', 'BMI', 'Height', 'Weight', 'Smoking', 'Alcohol_Use',
       'Helper'],
      dtype='object')


In [6]:
LBP_diagcode = LBP_all_data[['EMPI','Age','Race1','BMI','Code.prc','Code.dia','Helper']]
LBP_diagcode['Diagnosis_combined'] = str(0)
LBP_diagcode['LBP?'] = str(0)
print(LBP_diagcode.keys())

LBP_diagcode_reference = LBP_diagcode



Index(['EMPI', 'Age', 'Race1', 'BMI', 'Code.prc', 'Code.dia', 'Helper',
       'Diagnosis_combined', 'LBP?'],
      dtype='object')


/var/folders/m6/_674l8815692m04rgcb9n31h0000gn/T/ipykernel_49650/3317639091.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LBP_diagcode['Diagnosis_combined'] = str(0)
/var/folders/m6/_674l8815692m04rgcb9n31h0000gn/T/ipykernel_49650/3317639091.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LBP_diagcode['LBP?'] = str(0)


In [7]:
LBP_diagcode.loc[0,'Diagnosis_combined'] = LBP_diagcode.loc[0,'Code.dia']
for i in range(1,len(LBP_diagcode)):
    if (LBP_diagcode.loc[i,'EMPI'] == LBP_diagcode.loc[i-1,'EMPI']):
        if LBP_diagcode.loc[i,'Code.dia'] != LBP_diagcode.loc[i-1,'Code.dia']:
            LBP_diagcode.loc[i,'Diagnosis_combined'] = LBP_diagcode.loc[i,'Code.dia'] + ', ' + str(LBP_diagcode.loc[i-1,'Diagnosis_combined'])
            continue;
        else: 
            LBP_diagcode.loc[i,'Diagnosis_combined'] = LBP_diagcode.loc[i-1,'Diagnosis_combined']
            continue;
    else: 
        LBP_diagcode.loc[i,'Diagnosis_combined'] = LBP_diagcode.loc[i,'Code.dia']
        continue;

In [8]:
LBP_filtered = LBP_diagcode[LBP_diagcode['Helper'].notna()]
LBP_filtered_save = LBP_filtered

In [9]:
#CPT Code
#LBP only
incl_cpts_LBP = get_cpts_fromfile(path_inclusion_crit_LBP)
incl_cpts_text_LBP = '|'.join(incl_cpts_LBP.str.strip())

#scoliosis
incl_cpts_scoliosis = get_cpts_fromfile(path_inclusion_crit_scoliosis)
incl_cpts_text_scoliosis = '|'.join(incl_cpts_scoliosis.str.strip())

#ICD Code
#LBP only
incl_icds_LBP = get_icds_fromfile(path_inclusion_crit_LBP)
incl_icds_text_LBP = '|'.join(incl_icds_LBP.str.strip())

#scoliosis
incl_icds_scoliosis = get_icds_fromfile(path_inclusion_crit_scoliosis)
incl_icds_text_scoliosis = '|'.join(incl_icds_scoliosis.str.strip())

In [10]:
temp_df_LBP = pd.DataFrame
temp_df_LBP = LBP_filtered['Diagnosis_combined'].str.contains(incl_icds_text_LBP, regex=True)

LBP_filtered.index = range(len(LBP_filtered.index))

for i in range(len(temp_df_LBP)):
    if temp_df_LBP.iloc[i] == True:
        LBP_filtered.loc[i,'LBP?'] = 'LBP'

/var/folders/m6/_674l8815692m04rgcb9n31h0000gn/T/ipykernel_49650/484879454.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LBP_filtered.loc[i,'LBP?'] = 'LBP'


In [11]:
scoliosis_diagcode = scoliosis_all_data[['EMPI','Age','Race1','BMI','Code.prc','Code.dia','Helper']]
scoliosis_diagcode['Diagnosis_combined'] = str(0)
scoliosis_diagcode['scoliosis?'] = str(0)
print(scoliosis_diagcode.keys())

scoliosis_diagcode_reference = scoliosis_diagcode

Index(['EMPI', 'Age', 'Race1', 'BMI', 'Code.prc', 'Code.dia', 'Helper',
       'Diagnosis_combined', 'scoliosis?'],
      dtype='object')


/var/folders/m6/_674l8815692m04rgcb9n31h0000gn/T/ipykernel_49650/3493048040.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  scoliosis_diagcode['Diagnosis_combined'] = str(0)
/var/folders/m6/_674l8815692m04rgcb9n31h0000gn/T/ipykernel_49650/3493048040.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  scoliosis_diagcode['scoliosis?'] = str(0)


In [12]:
scoliosis_diagcode.loc[0,'Diagnosis_combined'] = scoliosis_diagcode.loc[0,'Code.dia']
for i in range(1,len(scoliosis_diagcode)):
    if (scoliosis_diagcode.loc[i,'EMPI'] == scoliosis_diagcode.loc[i-1,'EMPI']):
        if scoliosis_diagcode.loc[i,'Code.dia'] != scoliosis_diagcode.loc[i-1,'Code.dia']:
            scoliosis_diagcode.loc[i,'Diagnosis_combined'] = scoliosis_diagcode.loc[i,'Code.dia'] + ', ' + str(scoliosis_diagcode.loc[i-1,'Diagnosis_combined'])
            continue;
        else: 
            scoliosis_diagcode.loc[i,'Diagnosis_combined'] = scoliosis_diagcode.loc[i-1,'Diagnosis_combined']
            continue;
    else: 
        scoliosis_diagcode.loc[i,'Diagnosis_combined'] = scoliosis_diagcode.loc[i,'Code.dia']
        continue;

In [13]:
scoliosis_filtered = scoliosis_diagcode[scoliosis_diagcode['Helper'].notna()]
scoliosis_filtered_save = scoliosis_filtered

In [14]:
temp_df_scoliosis = pd.DataFrame
temp_df_scoliosis = scoliosis_filtered['Diagnosis_combined'].str.contains(incl_icds_text_scoliosis, regex=True)

scoliosis_filtered.index = range(len(scoliosis_filtered.index))

for i in range(len(temp_df_scoliosis)):
    if temp_df_scoliosis.iloc[i] == True:
        scoliosis_filtered.loc[i,'scoliosis?'] = 'scoliosis'

/var/folders/m6/_674l8815692m04rgcb9n31h0000gn/T/ipykernel_49650/1216154863.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  scoliosis_filtered.loc[i,'scoliosis?'] = 'scoliosis'


In [15]:
LBP_and_scoliosis = pd.concat([LBP_filtered,scoliosis_filtered])

In [16]:
LBP_and_scoliosis.index = range(len(LBP_and_scoliosis.index))
LBP_and_scoliosis['Helper'] = str(0)

In [17]:
LBP_and_scoliosis.sort_values(by=['EMPI'], inplace=True, ignore_index=True)
LBP_and_scoliosis_copy = LBP_and_scoliosis
LBP_and_scoliosis.head(10)

,EMPI,Age,Race1,BMI,Code.prc,Code.dia,Helper,Diagnosis_combined,LBP?,scoliosis?
0,100000152,81,White,27.02,74000,M41.9,0,M41.9,NaN,scoliosis
1,100000152,81,White,27.02,72100,G89.29,0,G89.29,LBP,NaN
2,100000243,92,Declined,24.73,72040,M54.9,0,M54.9,LBP,NaN
3,100001760,88,White,NaN,72040,G89.29,0,G89.29,LBP,NaN
4,100002085,89,White,27.92,72100,M41.9,0,M41.9,NaN,scoliosis
5,100003506,96,Declined,20.06,72100,M54.9,0,M54.9,LBP,NaN
6,100004276,92,Other,17.51,72040,G89.29,0,G89.29,LBP,NaN
7,100004276,92,Other,21.15,72100,M41.9,0,M41.9,NaN,scoliosis
8,100004276,92,Other,21.15,72100,M54.9,0,M54.9,LBP,NaN
9,100004610,98,White,NaN,72100,S30.0XXA,0,S30.0XXA,LBP,NaN


In [18]:
for i in range(1,len(LBP_and_scoliosis)):
    if LBP_and_scoliosis.loc[i,'EMPI'] == LBP_and_scoliosis.loc[i-1,'EMPI']:
        if LBP_and_scoliosis.loc[i,'Diagnosis_combined'] != LBP_and_scoliosis.loc[i-1,'Diagnosis_combined']:
            LBP_and_scoliosis.loc[i,'Diagnosis_combined'] = LBP_and_scoliosis.loc[i,'Diagnosis_combined'] + ', ' + str(LBP_and_scoliosis.loc[i-1,'Diagnosis_combined'])
        if LBP_and_scoliosis.loc[i,'LBP?'] != 'LBP':
            if LBP_and_scoliosis.loc[i-1,'LBP?'] == 'LBP':
                LBP_and_scoliosis.loc[i,'LBP?'] = 'LBP'
        if LBP_and_scoliosis.loc[i,'scoliosis?'] != 'scoliosis':
            if LBP_and_scoliosis.loc[i-1,'scoliosis?'] == 'scoliosis':
                LBP_and_scoliosis.loc[i,'scoliosis?'] = 'scoliosis'
    else:
        LBP_and_scoliosis.loc[i-1,'Helper'] = 'useful'

In [19]:
LBPnscoliosis_filtered = LBP_and_scoliosis.loc[LBP_and_scoliosis['Helper'] == 'useful']
LBPnscoliosis_filtered.reset_index(drop =True, inplace=True)
LBPnscoliosis_filtered['Both'] = str(0)

/var/folders/m6/_674l8815692m04rgcb9n31h0000gn/T/ipykernel_49650/390237996.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LBPnscoliosis_filtered['Both'] = str(0)


In [20]:
for i in range(0,len(LBPnscoliosis_filtered)):
    if LBPnscoliosis_filtered.loc[i,'LBP?'] == 'LBP':
        if LBPnscoliosis_filtered.loc[i,'scoliosis?'] == 'scoliosis':
            LBPnscoliosis_filtered.loc[i,'Both'] = 'Both'

In [21]:
LBPnscoliosis_filtered.head(20)

,EMPI,Age,Race1,BMI,Code.prc,Code.dia,Helper,Diagnosis_combined,LBP?,scoliosis?,Both
0,100000152,81,White,27.02,72100,G89.29,useful,"G89.29, M41.9",LBP,scoliosis,Both
1,100000243,92,Declined,24.73,72040,M54.9,useful,M54.9,LBP,NaN,0
2,100001760,88,White,NaN,72040,G89.29,useful,G89.29,LBP,NaN,0
3,100002085,89,White,27.92,72100,M41.9,useful,M41.9,NaN,scoliosis,0
4,100003506,96,Declined,20.06,72100,M54.9,useful,M54.9,LBP,NaN,0
5,100004276,92,Other,21.15,72100,M54.9,useful,"M54.9, M41.9, G89.29",LBP,scoliosis,Both
6,100004610,98,White,NaN,72100,S30.0XXA,useful,S30.0XXA,LBP,NaN,0
7,100006144,78,Black,23.33,72100,G89.29,useful,"G89.29, M40.12, M54.9, M40.12, M41.9, M40.209",LBP,scoliosis,Both
8,100006348,80,White,27,72040,G89.29,useful,G89.29,LBP,NaN,0
9,100007265,88,White,27.91,72100,M41.56,useful,"M41.56, G89.29",LBP,scoliosis,Both


In [22]:
LBP_only = LBPnscoliosis_filtered.loc[LBPnscoliosis_filtered['LBP?'] == 'LBP']
LBP_only = LBP_only[LBP_only['Both'] != 'Both']
scoliosis_only = LBPnscoliosis_filtered.loc[LBPnscoliosis_filtered['scoliosis?'] == 'scoliosis']
scoliosis_only = scoliosis_only[scoliosis_only['Both'] != 'Both']
LBPscoliosis_Both_only = LBPnscoliosis_filtered.loc[LBPnscoliosis_filtered['Both'] == 'Both']

In [23]:
LBP_and_scoliosis_total = pd.concat([LBP_diagcode,scoliosis_diagcode])
LBP_and_scoliosis_total.drop_duplicates(subset=['EMPI'],inplace=True)
len(LBP_and_scoliosis_total)

4659

In [24]:
print('Number of Total Patients(LBP+scoliosis):  ' + str(len(LBP_and_scoliosis_total)-1))
print('Number of LBP only:  ' + str(len(LBP_only)))
print('Number of scoliosis only:  ' + str(len(scoliosis_only)))
print('Number of Both only:  ' + str(len(LBPscoliosis_Both_only)))

#LBP_filtered = LBP
#scoliosis_filtered = scoliosis
#LBP & scoliosis filtered = LBPnscoliosis_filtered


Number of Total Patients(LBP+scoliosis):  4658
Number of LBP only:  3666
Number of scoliosis only:  338
Number of Both only:  654


In [25]:
LBP_patients = LBP_filtered.sort_values(by=['EMPI'], ignore_index=True)
LBP_patients.head()

,EMPI,Age,Race1,BMI,Code.prc,Code.dia,Helper,Diagnosis_combined,LBP?
0,100000152,81,White,27.02,72100,G89.29,useful,G89.29,LBP
1,100000243,92,Declined,24.73,72040,M54.9,useful,M54.9,LBP
2,100001760,88,White,NaN,72040,G89.29,useful,G89.29,LBP
3,100003506,96,Declined,20.06,72100,M54.9,useful,M54.9,LBP
4,100004276,92,Other,17.51,72040,G89.29,useful,G89.29,LBP


In [26]:
scoliosis_csv_path = os.path.join(filepath_root,'LBP_scoliosis_patients.csv')
LBPscoliosis_Both_only.to_csv(scoliosis_csv_path)

In [27]:
scoliosis_only_csv_path = os.path.join(filepath_root,'scoliosis_patients.csv')
scoliosis_filtered.to_csv(scoliosis_only_csv_path)

In [28]:
LBPscoliosis_Both_only.head()

,EMPI,Age,Race1,BMI,Code.prc,Code.dia,Helper,Diagnosis_combined,LBP?,scoliosis?,Both
0,100000152,81,White,27.02,72100,G89.29,useful,"G89.29, M41.9",LBP,scoliosis,Both
5,100004276,92,Other,21.15,72100,M54.9,useful,"M54.9, M41.9, G89.29",LBP,scoliosis,Both
7,100006144,78,Black,23.33,72100,G89.29,useful,"G89.29, M40.12, M54.9, M40.12, M41.9, M40.209",LBP,scoliosis,Both
9,100007265,88,White,27.91,72100,M41.56,useful,"M41.56, G89.29",LBP,scoliosis,Both
13,100008826,72,White,21.46,72100,M51.26,useful,"M51.26, G89.29, M41.9",LBP,scoliosis,Both
